In [9]:
# Imports
import numpy as np
import random as rd
import math
import IOHexperimenter as IOH
from IOHexperimenter import IOH_function

#setting the random seed
rd.seed()

In [93]:
class Simple_EA:
    
    def __init__(self, budget):
        self.budget = budget
        self.f_opt = np.Inf
        self.x_opt = None
        self.d = 1
                
        # Parameter setting
        self.mu = 15 #number of parents
        self.lamda = 100 #number of offspring
        self.pc = 0.8 #recombination probability
        self.tau = 1 / math.sqrt(self.d)
        self.sigma = 10/6
        
    def initialize_individual(self):
        individual = np.empty((self.d,1))
        for i in range (0,self.d):
            individual[i]= rd.uniform(-5, 5)
        return individual

    def initialize_population(self):
        population = np.zeros((self.mu,self.d))
        for k in range(0,self.mu):
            population[k] = np.transpose(self.initialize_individual())
        return population

    def recombine(self,pop):
        offspring = np.zeros((self.d))
        p1 = pop[rd.randrange(0,len(pop))]
        p2 = pop[rd.randrange(0,len(pop))]
        if rd.uniform(0,1) < self.pc:
            for k in range (0,self.d):
                random_number = rd.uniform(0,1)
                if random_number < 0.5:
                    offspring[k] = p1[k]
                else:
                    offspring[k] = p2[k]
        else:
            offspring = p1
        return offspring

    def update_sigma(self):
        self.sigma = self.sigma * math.exp(self.tau * np.random.normal(0,1))
        return self.sigma

    def singlestep_mutate (self,ind):
        mutant = np.zeros(len(ind))
        for j in range (0, len(ind)):
             result = ind[j] + self.sigma * np.random.normal(0,1)
             # check boundaries first
             if result < -5:
                mutant[j] = -5
             elif result > 5:
                mutant[j] = 5
             # if result is within boundaries, change j-th mutant characteristic to result
             else:
                mutant[j] = result

        return mutant
  
    def __call__(self, func):
        
        evalcount = 0
        self.d = func.number_of_variables
        eval_budget = self.budget * self.d
        
        #Initialization
        xp = self.initialize_population() #parent population
        f = np.zeros((self.mu,1)) #parents fitness
        xo = np.zeros((self.lamda,self.d)) #offspring population
        fo = np.zeros((self.lamda,1)) #offspring fitness
        
        for i in range (0,self.mu):
            f[i] = func(xp[i])
            evalcount = evalcount +1
        
        # Evaluation loop
        while evalcount < eval_budget and func.final_target_hit == False :
            self.sigma = self.update_sigma()
            
            for i in range(0,self.lamda):
                offspring = self.recombine(xp)
                xo[i] = self.singlestep_mutate(offspring)
                fo[i] = func(xo[i])
                evalcount = evalcount +1
                
            # Selection
            m = np.concatenate((xo, fo), axis=1) # Create joint matrix of individuals and their fitness values
            sorted_m = m[np.argsort(m[:, self.d])] # sort based on fitness (lowest values highest ranking)
            xpnew = sorted_m[0:self.mu,0:self.d] # copy best individuals and remove fitness values

            xp = xpnew # set new population for next generation
            f = sorted_m[0:self.mu,self.d] # copy fitness values of new generation

            self.x_opt = xpnew[0]
            self.f_opt = f[0]

        return self.x_opt, self.f_opt

In [86]:
list = []
for i in range (1,25):
    list.append(i)

In [87]:
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()

exp.initialize_BBOB(list,[1],[2,5,20], 1)
exp.set_logger_location("experiment7", "run")

In [95]:
exp([Simple_EA(10000)])

Running single-threaded
1 1715 8.700259962071172e-09
1 50015 2.5431594640002743e-07
1 200015 0.3989321136792331
2 20015 91786.78661691122
2 50015 764781.1738962715
2 200015 3851893.006061066
3 20015 21.01799443817913
3 50015 129.3586312664454
3 200015 1169.4275295852792
4 20015 21.738943482194472
4 50015 89.248399893284
4 200015 292.3523434972365
5 115 0.0
5 115 0.0
5 200015 18.32980710832436
6 20015 5.802209572317167
6 50015 404.91715431297814
6 200015 507069.5645872644
7 20015 97.83977110247267
7 50015 173.7413015546301
7 200015 590.1181630405185
8 20015 5.635591865225445
8 50015 2732.378570750684
8 200015 111950.51504091856
9 20015 6.377770037556524
9 50015 4823.495368060562
9 200015 166860.04628894522
10 20015 162572.46743129927
10 50015 195583.32928193797
10 200015 837404.9036072805
11 20015 875.2792652326605
11 50015 19728.684690072412
11 200015 352.07340422530586
12 20015 844.6285957940108
12 50015 1959878.5061518627
12 200015 241844847.4035083
13 20015 122.82387488768235
13 500

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [45]:
np.random.normal(0,1)

0.3687509655995906

In [20]:
rd.uniform(0,15)

12.026611990799731

In [55]:
ind = np.array([4.8, -4.9, 2.004])

In [56]:
len(ind)

3

In [70]:
mutant = np.zeros(len(ind))
for j in range (0, len(ind)):
     result = ind[j] + 0.5 * np.random.normal(0,1)
     # check boundaries first
     if result < -5:
        mutant[j] = -5
     elif result > 5:
        mutant[j] = 5
     # if result is within boundaries, change j-th mutant characteristic to result
     else:
        mutant[j] = result

In [71]:
mutant

array([ 5.        , -4.95380178,  1.36805   ])

In [84]:
lowerbound = -5
upperbound = 5
individual = np.empty((5,1))
for i in range (0,5):
    individual[i]= rd.uniform(lowerbound, upperbound)
individual

array([[ 4.54828113],
       [ 0.15893617],
       [ 0.2418503 ],
       [-2.96042667],
       [-1.59883677]])